# The hard way 

First, I'll show you the hard way, if you want to do all of this from scratch 

In [1]:
import torch 
from torch_geometric.datasets import KarateClub

In [2]:
# Just a toy dataset 
g = KarateClub().data
edge_index = g.edge_index
NUM_NODES = g.num_nodes 

# Top row is source nodes, bottom row is destination nodes
print(edge_index)

tensor([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,
          1,  1,  1,  1,  1,  1,  1,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  3,
          3,  3,  3,  3,  3,  4,  4,  4,  5,  5,  5,  5,  6,  6,  6,  6,  7,  7,
          7,  7,  8,  8,  8,  8,  8,  9,  9, 10, 10, 10, 11, 12, 12, 13, 13, 13,
         13, 13, 14, 14, 15, 15, 16, 16, 17, 17, 18, 18, 19, 19, 19, 20, 20, 21,
         21, 22, 22, 23, 23, 23, 23, 23, 24, 24, 24, 25, 25, 25, 26, 26, 27, 27,
         27, 27, 28, 28, 28, 29, 29, 29, 29, 30, 30, 30, 30, 31, 31, 31, 31, 31,
         31, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 33, 33, 33, 33, 33,
         33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33],
        [ 1,  2,  3,  4,  5,  6,  7,  8, 10, 11, 12, 13, 17, 19, 21, 31,  0,  2,
          3,  7, 13, 17, 19, 21, 30,  0,  1,  3,  7,  8,  9, 13, 27, 28, 32,  0,
          1,  2,  7, 12, 13,  0,  6, 10,  0,  6, 10, 16,  0,  4,  5, 16,  0,  1,
          2,  3,  0,  2, 30, 32, 33,  2, 33,  0,  4

In [3]:
# High-performance random walk generator 
from torch_cluster.rw import random_walk

# Need to add self loops, otherwise if rw generator gets to a terminal 
# node, when it tries to get the next edge probablility (1/num_edges) 
# it hits a divide by zero error and crashes
from torch_geometric.utils import add_remaining_self_loops
edge_index,_ = add_remaining_self_loops(edge_index)

WALK_LEN = 3
start_nodes = torch.arange(NUM_NODES)
src = edge_index[0]
dst = edge_index[1]
rws = random_walk(src, dst, start_nodes, WALK_LEN)
print(rws[:10])

tensor([[ 0,  3,  1,  2],
        [ 1, 30,  8, 32],
        [ 2,  8,  2,  1],
        [ 3,  7,  1, 30],
        [ 4, 10, 10,  5],
        [ 5,  0,  2, 28],
        [ 6,  0, 10,  4],
        [ 7,  0,  6,  0],
        [ 8, 33, 27, 23],
        [ 9, 33, 33, 28]])


In [4]:
# Threw together simple skipgram model
# (note: i didn't bother with negative samples, so this runs the 
# original word2vec algorithm technically)
from skipgram import SkipGram
from torch.optim import Adam

EPOCHS = 1000
ENC_DIM = 16

model = SkipGram(NUM_NODES, ENC_DIM)
opt = Adam(model.parameters(), lr=0.01)
for e in range(EPOCHS):
    opt.zero_grad()
    rws = random_walk(*edge_index, torch.arange(NUM_NODES), WALK_LEN)
    batch, rw = rws[:,0], rws[:,1:]
    loss = model(batch, rw) # Calls model.forward
    loss.backward()
    opt.step()

with torch.no_grad():
    src = model.embed(edge_index[0][:10])
    dst = model.embed(edge_index[1][:10])
    rnd_src = model.embed(torch.randint(0, NUM_NODES-1, (10,)))
    rnd_dst = model.embed(torch.randint(0, NUM_NODES-1, (10,)))

print("Dot product of neighbors:")
print(torch.sigmoid((src * dst).sum(dim=1)))

print("Dot product of non-neighbors")
print(torch.sigmoid((rnd_src * rnd_dst).sum(dim=1)))


Dot product of neighbors:
tensor([0.9825, 0.5487, 0.9946, 0.9381, 0.7961, 0.8068, 0.9461, 0.4466, 0.9523,
        0.9994])
Dot product of non-neighbors
tensor([0.1395, 0.0416, 0.6067, 0.0134, 0.9786, 0.9952, 0.8769, 0.9971, 0.9964,
        0.4502])


This does pretty well, but it doesn't have a lot of the fancy optimizations of the torch_geo Node2Vec model. Plus, that model has everything from random walks to inference built in already. So let's see how that runs... 

# The easy way

In [5]:
from torch_geometric.nn.models import Node2Vec

n2v = Node2Vec(edge_index, ENC_DIM, WALK_LEN, WALK_LEN)
opt = Adam(n2v.parameters(), lr=0.01)

for e in range(EPOCHS):
    opt.zero_grad()
    pos_sample = n2v.pos_sample(torch.arange(NUM_NODES))
    neg_sample = n2v.neg_sample(torch.arange(NUM_NODES))
    loss = n2v.loss(pos_sample, neg_sample)
    loss.backward()
    opt.step()

with torch.no_grad():
    src = n2v(edge_index[0][:10])
    dst = n2v(edge_index[1][:10])
    rnd_src = n2v(torch.randint(0, NUM_NODES-1, (10,)))
    rnd_dst = n2v(torch.randint(0, NUM_NODES-1, (10,)))

print("Dot product of neighbors:")
print(torch.sigmoid((src * dst).sum(dim=1)))

print("Dot product of non-neighbors")
print(torch.sigmoid((rnd_src * rnd_dst).sum(dim=1)))

Dot product of neighbors:
tensor([0.8469, 0.6914, 0.7759, 0.7902, 0.7484, 0.7404, 0.8190, 0.6893, 0.7557,
        0.8221])
Dot product of non-neighbors
tensor([0.3816, 0.6139, 0.7081, 0.4665, 0.4770, 0.9148, 0.5139, 0.2607, 0.7462,
        0.3050])


Note that this is a small-ish dataset, so some of the "non-edges" may be edges just by chance... I didn't really validate this to try to push it out quickly. 